# Molecular Dynamics 3.5 

Now that we have a code with multiple "atoms," let's have them interact with each other. One simple and intuitive interaction is a hard-sphere (perfectly elastic) collision. Let's do this.

The initial set-up is the same as before:

In [1]:
import turtle
import random
import math

In [2]:
random.seed()

In [3]:
window = turtle.Screen()
window.title('Molecular Dynamics 3.5')
window.clear()

In [4]:
# We will want to store these numbers in a variable
height = window.window_height()
width = window.window_width()

In [5]:
# Number of atoms we want
num_atoms = 20

# This initializes an empty list
atoms = []

# Now use a loop to initialize a new atom
# and do it num_atoms times.
for i in range(num_atoms):
    atoms.append(turtle.Turtle())

In [6]:
# Max velocity we want
max_velocity = 80.0

# Initailize an empty list
vel = []

# Now loop over the number of atoms
for i in range(num_atoms):
    vel.append([random.uniform(-1,1)*max_velocity, random.uniform(-1,1)*max_velocity])

OK, now that that's out of the way, let's put in some new stuff that allows us to have our atoms interact. First, we need a function that calculates the distance between our two atoms. We'll use the well-known distance formula and the positions of the center of each atom.

Below is a function that takes two atoms as inputs and returns the distance between the two centers.

In [7]:
# Lets name our function
def distance(atom1, atom2):
    
    # Inputs are the atoms, which contain
    # (x,y) coordinate pairs.
    # Distance is sqrt((x1-x2)^2 + (y1-y2)^2)
    
    # Lets get the coordinates of atom1
    (x1,y1) = atom1.pos()
    
    # Lets get the coordinates of atom2
    (x2,y2) = atom2.pos()

    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d
    

The cell below is similar to one we've seen before. It gives the atoms random positions on the screen. However, there's an important modification we've had to make. If the atoms are hard spheres, it doesn't make sense for them to be on top of each other. Therefore, as we position the atoms, we need to check to make sure they don't "stack." Fortunately, we just wrote a distance function that we can use to check this.

In [8]:
# Variables to hold things we want to be constant
atom_radius = 20

# Scaling factor here is so we don't get our 
# initial positions stuck on the edge.
scaling_factor = 0.8

# We need to loop over each atom.
for i in range(num_atoms):
   
    # Draw the atom in the proper shape
    atoms[i].shape('circle')
    atoms[i].shapesize(atom_radius/10.0)
    atoms[i].color((random.random(),random.random(),random.random()))
    atoms[i].penup()
    #The following line moves the atom to a random position on the screen.
    atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, random.uniform(-1,1)*height/2.0 * scaling_factor)
  
    # Lets make sure this atom isn't on top of another atom
    # Lets get the minimum distance to all other atoms
    # It suffices to look at the distance to the closest atom.
    # Let's call this distance d.
    
    d = 1000000.0 #Just initialize d to some big number
    for j in range(0,i):
        d = min(d, distance(atoms[i],atoms[j]))
    
    #Now, keep redrawing the atom until it's sufficiently far
    #away from all other atoms. If it's already non-overlapping,
    #this loop will never execute.
    while d < 2.0*atom_radius:
        # Reset d
        d = 1000000.0
        # Choose a new random spot
        atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, 
                      random.uniform(-1,1)*height/2.0 * scaling_factor)
        #recalculate d
        for j in range(0,i):
            d = min(d, distance(atoms[i],atoms[j]))
        
    # Turtles can be very slow. This is a semi-fix to tell
    # turtles not to update the screen with every change,
    # but rather wait till a set of updates are done and 
    # then update the screen.
    atoms[i].tracer(0,0)
    turtle.update()
    

Checking for a bounce is something that we're going to do a lot, so let's write a function that can do it for us. There are many ways to go about doing this. The one we wrote takes the lists of atoms (positions) and velocities. Using the position list, this first checks to see if any atoms have collided. If they have, it updates the velocities of the two colliding atoms.

In [9]:
# Lets name our function
def bounce_check(a_list, v_list):
  
    # This will help us not double count atoms
    # We need to keep track of this index by hand
    # because we are using a for loop later.
    i = 1

    # We need to check for every atom
    # Using a for loop here isn't required, but it 
    # makes the code easier.
    for atom1 in a_list:
        
        # This will help us not double count
        # (same argument as above)
        j = i
        
        # Compute the distance to every atom ahead of the current one
        for atom2 in a_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                #print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # Now calculate the new velocities
                # First, calculate the intermediate quantity (v2-v1).R (dot product)
                inter = ((v_list[j][0]-v_list[i-1][0])*(x2-x1) + (v_list[j][1]-v_list[i-1][1])*(y2-y1))/(d**2)
                #v1R = v_list[i-1][0]*(x1-x2) + v_list[i-1][1]*(y1-y2) #projection of first atom velocity onto R
                #v2R = v_list[j][0]*(x1-x2) + v_list[j][1]*(y1-y2) #projection of 2nd atom velocity onto R
                
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] + inter * (x2-x1)
                v_list[i-1][1] = v_list[i-1][1] + inter * (y2-y1)
                v_list[j][0] = v_list[j][0] - inter * (x2-x1)
                v_list[j][1] = v_list[j][1] - inter * (y2-y1)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1
    
    #return the list of new velocities
    return v_list

Now we're ready to simulate.

In [10]:
# The amount of time each iteration moves us forward
dt = 0.05

# Max number of steps we want to take
max_steps = 10000


for step in range(max_steps):

    # This for loop checks if we've hit a wall
    for j in range(num_atoms):
    
        # Get the current position and velocity of the atom
        vx,vy = vel[j]    
        x,y = atoms[j].pos()
    
        # Check if moving left or right will put our atom beyond the wall
        if abs(x + dt * vel[j][0]) >= width/2.0 - atom_radius:
        
            # We have moved too far right or left, so flip the x_vel
            vel[j][0] = -vel[j][0]           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(y + dt * vel[j][1]) >= height/2.0 - atom_radius:
        
            # We have moved too far up or down, so flip the y_vel
            vel[j][1] = -vel[j][1]
        
    #Use our bounce_check function to update the velocities, if needed.
    vel = bounce_check(atoms,vel)
    
    #Now that everything checks out, go ahead and move our atoms
    for j in range(num_atoms):
        x,y = atoms[j].pos()
        atoms[j].goto(x + dt*vel[j][0], y + dt*vel[j][1])
        
        
    # Tell turtles we are done updating and to redraw
    turtle.update()
    

It works!

Always clean up.

In [11]:
window.bye()